In [1]:
import sys,subprocess
pkgs=["autogluon.tabular==1.4.0","catboost==1.2.8","tqdm"]
subprocess.check_call([sys.executable,"-m","pip","install","-q"]+pkgs)


0

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from catboost import CatBoostRegressor,Pool
from catboost.utils import get_gpu_device_count
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

from pathlib import Path
import pandas as pd

path = Path(".")

train = pd.read_csv(path/"train.tsv", sep="\t")
test = pd.read_csv(path/"test.tsv", sep="\t")



In [17]:
train.columns

Index(['id', 'name', 'coordinates', 'category', 'address', 'target',
       'traffic_300m', 'homes_300m', 'works_300m', 'female_300m',
       ...
       'doramas_1000m', 'computer_components_1000m', 'humor_1000m',
       'car_market_1000m', 'no_higher_education_1000m',
       'goods_for_moms_and_babies_1000m', 'age_25-34_1000m', 'male_1000m',
       'phone_repair_1000m', 'mean_income_1000m'],
      dtype='object', length=286)

In [15]:
test

,id,name,coordinates,category,address,traffic_300m,homes_300m,works_300m,female_300m,train_ticket_order_300m,...,doramas_1000m,computer_components_1000m,humor_1000m,car_market_1000m,no_higher_education_1000m,goods_for_moms_and_babies_1000m,age_25-34_1000m,male_1000m,phone_repair_1000m,mean_income_1000m
0,21472,Счастье,"[37.533334, 55.790246]",candy_shop,"Ходынский бул., 4, Москва",62672,4709.110524,4298.125296,38987.0,961.0,...,1706.0,5764.0,32190.0,12370.0,265916.0,221.0,226914.0,399582.0,2255.0,122883.795473
1,9837,O'STIN,"[37.886829, 55.751627]",baby_clothes,"Носовихинское ш., 45, Реутов",110226,12987.989255,15235.256665,96081.0,1346.0,...,1266.0,3930.0,17037.0,9044.0,233487.0,102.0,214465.0,320022.0,1801.0,113878.735454
2,41791,Дровосек,"[37.474419, 55.863549]",barbershop,"Беломорская ул., 18А, корп. 2, Москва",81080,9575.248571,9463.322898,57147.0,1506.0,...,1915.0,6549.0,32133.0,13745.0,298014.0,305.0,253803.0,431408.0,2426.0,119179.257929
3,18441,Милти,"[37.532347, 55.607307]",cooking,"Новоясеневский просп., 7, Москва",131046,21152.067433,22155.559230,113526.0,1777.0,...,2345.0,5683.0,26201.0,12758.0,318360.0,233.0,245132.0,427442.0,2053.0,110069.641621
4,49348,Белая лебедь,"[37.559468, 55.863468]",laundry,"Бескудниковский бул., 13, Москва",87605,18499.759458,18868.131714,68314.0,1201.0,...,2868.0,8647.0,36731.0,17519.0,465743.0,379.0,409782.0,657632.0,3121.0,114652.304715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9271,30097,Uhostels,"[37.382774, 55.711967]",hostel,"Можайское ш., 165, рабочий посёлок Новоивановское",36378,2968.320703,1891.917847,16430.0,293.0,...,780.0,2220.0,11360.0,7518.0,152220.0,67.0,141823.0,251806.0,1014.0,115239.815808
9272,21993,Детский мир,"[37.493409, 55.424235]",toys,"Ленинградская ул., 21, Подольск",94187,19244.142246,17823.348711,87563.0,1116.0,...,1616.0,3917.0,16118.0,10627.0,305390.0,176.0,237858.0,391213.0,1753.0,104614.619289
9273,43919,Ice Curly,"[37.670418, 55.780504]",hair_salon,"Гаврикова ул., 2/38, Москва",178064,19000.549439,20373.539460,127901.0,3529.0,...,9144.0,21962.0,111717.0,43129.0,1166066.0,544.0,920617.0,1509700.0,8968.0,115478.531317
9274,46598,Мясницкий ряд,"[37.464845, 55.785135]",butcher_shop,"ул. Маршала Тухачевского, 49, Москва",49927,11251.957179,9505.878594,36235.0,655.0,...,1868.0,4650.0,23989.0,10899.0,254020.0,407.0,209882.0,373401.0,1817.0,117662.523365


In [18]:
target_col = "target"  # или как называется таргет в новом датасете
feature_cols = [c for c in train.columns if c != target_col and c in test.columns]

X = train[feature_cols]
y = train[target_col].values
X_test = test[feature_cols]

In [22]:
print("До автофичей:", X.shape)          # (41105, N_исходных_фич)
print("Число исходных фич:", X.shape[1])


До автофичей: (41105, 285)
Число исходных фич: 285


In [19]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

fg = AutoMLPipelineFeatureGenerator(
    enable_numeric_features=True,
    enable_categorical_features=True,
    enable_datetime_features=True,
    enable_text_special_features=True,
    enable_text_ngram_features=True,
    enable_raw_text_features=False,
)

X_tr_feat = fg.fit_transform(X=X, y=y)
X_te_feat = fg.transform(X_test)

print(X_tr_feat.shape, X_te_feat.shape)
print(X_tr_feat.columns[:30])



Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    440410.22 MB
	Train Data (Original)  Memory Usage: 101.85 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['address']
			CountVectorizer fit with vocabulary size = 792
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data

(41105, 916) (9276, 916)
Index(['id', 'traffic_300m', 'homes_300m', 'works_300m', 'female_300m',
       'train_ticket_order_300m', 'mortgage_300m', 'recipes_300m',
       'online_shops_300m', 'manga_300m', 'children_goods_300m',
       'language_courses_300m', 'commercial_real_estate_purchase_300m',
       'grocery_stores_300m', 'preschool_300m', 'cultural_leisure_events_300m',
       'banking_services_300m', 'no_children_300m', 'mid_class_cars_300m',
       'home_appliances_300m', 'beauty_and_health_devices_300m',
       'computer_games_300m', 'cosplay_300m', 'unemployed_300m',
       'pizza_delivery_300m', 'science_and_technology_300m', 'married_300m',
       'mobile_phones_300m', 'events_300m', 'deposits_300m'],
      dtype='object')


In [23]:
print("После автофичей:", X_tr_feat.shape)   # (41105, 916)
print("Число фич после генерации:", X_tr_feat.shape[1])


После автофичей: (41105, 916)
Число фич после генерации: 916


In [20]:
from catboost.utils import get_gpu_device_count
import catboost, sys
print("python:", sys.executable)
print("catboost version:", catboost.__version__)
print("GPU devices seen by catboost:", get_gpu_device_count())


python: /venv/main/bin/python
catboost version: 1.2.8
GPU devices seen by catboost: 2


In [27]:
import numpy as np

cat_cols = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("category", "object")
]

for c in cat_cols:
    X_tr_feat[c] = X_tr_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")
    if c in X_te_feat.columns:
        X_te_feat[c] = X_te_feat[c].astype(str).replace({"nan": "missing"}).fillna("missing")

print("Категориальных колонок:", len(cat_cols))


Категориальных колонок: 4


In [28]:
cat_features = [
    c for c in X_tr_feat.columns
    if str(X_tr_feat[c].dtype) in ("object",)  # теперь всё строковое
]
print(len(cat_features), "cat_features")


4 cat_features


In [29]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
import numpy as np

idx = np.arange(len(X_tr_feat))
np.random.seed(42)
np.random.shuffle(idx)
small_idx = idx[:1000]

X_small = X_tr_feat.iloc[small_idx]
y_small = y[small_idx]

train_pool = Pool(X_small, y_small, cat_features=cat_features)

model_gpu_test = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    iterations=300,
    learning_rate=0.05,
    depth=6,
    task_type="GPU",
    devices="0",
    verbose=50,
    random_seed=42,
)

print("=== GPU sanity fit (300 iters on 1000 rows) ===")
model_gpu_test.fit(train_pool)


=== GPU sanity fit (300 iters on 1000 rows) ===
0:	learn: 1.1599236	total: 13.5ms	remaining: 4.03s
50:	learn: 1.0597656	total: 510ms	remaining: 2.49s
100:	learn: 0.9966601	total: 1s	remaining: 1.97s
150:	learn: 0.9436507	total: 1.47s	remaining: 1.45s
200:	learn: 0.8919346	total: 1.95s	remaining: 958ms
250:	learn: 0.8496613	total: 2.4s	remaining: 468ms
299:	learn: 0.8015994	total: 2.83s	remaining: 0us


In [31]:
import pandas as pd
import numpy as np

for c in X_tr_feat.columns:
    if X_tr_feat[c].dtype == "object":
        X_tr_feat[c] = X_tr_feat[c].fillna("missing").astype(str)
        if c in X_te_feat.columns:
            X_te_feat[c] = X_te_feat[c].fillna("missing").astype(str)
    else:
        X_tr_feat[c] = pd.to_numeric(X_tr_feat[c], errors="coerce")
        if c in X_te_feat.columns:
            X_te_feat[c] = pd.to_numeric(X_te_feat[c], errors="coerce")

cat_features = [
    i for i, c in enumerate(X_tr_feat.columns)
    if X_tr_feat[c].dtype == "object"
]

print("cat_features count:", len(cat_features))

cat_features count: 4


In [ ]:
'''надо поменять вообще было функцию на MAE для этой соревы но я проебался малек'''

In [32]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from tqdm.auto import tqdm
import numpy as np

def rmse_func(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

N_SPLITS = 5
N_ITER = 3000
DEPTH = 8
OD_WAIT = 300
TASK_TYPE = "GPU"

X = X_tr_feat
y_arr = y

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof = np.zeros(len(X))
test_pred = np.zeros(len(X_te_feat))
scores = []

for fold, (tr_idx, val_idx) in enumerate(tqdm(kf.split(X), total=N_SPLITS, desc="Folds"), 1):
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y_arr[tr_idx], y_arr[val_idx]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    params = dict(
        loss_function="RMSE",
        eval_metric="RMSE",
        iterations=N_ITER,
        learning_rate=0.01,
        depth=DEPTH,
        l2_leaf_reg=3.0,
        random_seed=42 + fold,
        verbose=200,
        od_type="Iter",
        od_wait=OD_WAIT,
        task_type=TASK_TYPE,
    )

    if TASK_TYPE == "GPU":
        params["devices"] = "0"

    print(f"\n=== Fold {fold}/{N_SPLITS} ===")
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    val_pred = model.predict(X_val)
    oof[val_idx] = val_pred
    rmse = rmse_func(y_val, val_pred)
    scores.append(rmse)
    print(f"Fold {fold} RMSE: {rmse:.5f}")

    bs = 1024
    fold_test = np.zeros(len(X_te_feat))
    for s in tqdm(range(0, len(X_te_feat), bs), leave=False, desc=f"Predict fold {fold}"):
        e = s + bs
        fold_test[s:e] = model.predict(X_te_feat.iloc[s:e])
    test_pred += fold_test / N_SPLITS

oof_rmse = rmse_func(y_arr, oof)
print("\nfold_rmse:", scores)
print("mean_rmse:", np.mean(scores))
print("oof_rmse:", oof_rmse)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]


=== Fold 1/5 ===
0:	learn: 1.1961353	test: 1.1567529	best: 1.1567529 (0)	total: 60.7ms	remaining: 3m 1s
200:	learn: 1.0681650	test: 1.0396617	best: 1.0396617 (200)	total: 11.6s	remaining: 2m 41s
400:	learn: 1.0506453	test: 1.0278411	best: 1.0278411 (400)	total: 23.3s	remaining: 2m 30s
600:	learn: 1.0429004	test: 1.0241462	best: 1.0241462 (600)	total: 34.6s	remaining: 2m 18s
800:	learn: 1.0370540	test: 1.0220961	best: 1.0220961 (800)	total: 45.8s	remaining: 2m 5s
1000:	learn: 1.0313323	test: 1.0203050	best: 1.0203050 (1000)	total: 57s	remaining: 1m 53s
1200:	learn: 1.0259834	test: 1.0190701	best: 1.0190701 (1200)	total: 1m 8s	remaining: 1m 42s
1400:	learn: 1.0213685	test: 1.0183588	best: 1.0183525 (1396)	total: 1m 18s	remaining: 1m 30s
1600:	learn: 1.0170179	test: 1.0179110	best: 1.0179110 (1600)	total: 1m 29s	remaining: 1m 18s
1800:	learn: 1.0130720	test: 1.0175272	best: 1.0175096 (1793)	total: 1m 41s	remaining: 1m 7s
2000:	learn: 1.0092340	test: 1.0171160	best: 1.0171160 (2000)	total

Predict fold 1:   0%|          | 0/10 [00:00<?, ?it/s]


=== Fold 2/5 ===
0:	learn: 1.1864868	test: 1.1966088	best: 1.1966088 (0)	total: 57.9ms	remaining: 2m 53s
200:	learn: 1.0625701	test: 1.0634628	best: 1.0634628 (200)	total: 11.7s	remaining: 2m 43s
400:	learn: 1.0465040	test: 1.0490837	best: 1.0490837 (400)	total: 23.2s	remaining: 2m 30s
600:	learn: 1.0383157	test: 1.0441042	best: 1.0441042 (600)	total: 34.9s	remaining: 2m 19s
800:	learn: 1.0325374	test: 1.0416734	best: 1.0416719 (798)	total: 46.3s	remaining: 2m 7s
1000:	learn: 1.0274891	test: 1.0395436	best: 1.0395391 (999)	total: 57.7s	remaining: 1m 55s
1200:	learn: 1.0229930	test: 1.0382931	best: 1.0382931 (1200)	total: 1m 8s	remaining: 1m 43s
1400:	learn: 1.0187774	test: 1.0374908	best: 1.0374908 (1400)	total: 1m 20s	remaining: 1m 31s
1600:	learn: 1.0140191	test: 1.0362455	best: 1.0362455 (1600)	total: 1m 31s	remaining: 1m 19s
1800:	learn: 1.0103969	test: 1.0355906	best: 1.0355842 (1794)	total: 1m 42s	remaining: 1m 8s
2000:	learn: 1.0069181	test: 1.0350323	best: 1.0350323 (2000)	tot

Predict fold 2:   0%|          | 0/10 [00:00<?, ?it/s]

...


=== Fold 4/5 ===
0:	learn: 1.1777454	test: 1.2313605	best: 1.2313605 (0)	total: 56.7ms	remaining: 2m 50s
200:	learn: 1.0542061	test: 1.0938335	best: 1.0938335 (200)	total: 11.5s	remaining: 2m 40s
400:	learn: 1.0390503	test: 1.0779432	best: 1.0779432 (400)	total: 22.7s	remaining: 2m 27s
600:	learn: 1.0319989	test: 1.0729167	best: 1.0729167 (600)	total: 34s	remaining: 2m 15s
800:	learn: 1.0261835	test: 1.0694670	best: 1.0694670 (800)	total: 45.3s	remaining: 2m 4s
1000:	learn: 1.0212496	test: 1.0670310	best: 1.0670310 (1000)	total: 56.4s	remaining: 1m 52s
1200:	learn: 1.0166460	test: 1.0656923	best: 1.0656923 (1200)	total: 1m 7s	remaining: 1m 41s
1400:	learn: 1.0121602	test: 1.0645263	best: 1.0645254 (1398)	total: 1m 18s	remaining: 1m 29s
1600:	learn: 1.0076293	test: 1.0630999	best: 1.0630971 (1597)	total: 1m 29s	remaining: 1m 18s
1800:	learn: 1.0034396	test: 1.0620857	best: 1.0620848 (1798)	total: 1m 41s	remaining: 1m 7s
2000:	learn: 0.9997029	test: 1.0611826	best: 1.0611789 (1999)	tota

Predict fold 4:   0%|          | 0/10 [00:00<?, ?it/s]


=== Fold 5/5 ===
0:	learn: 1.1899427	test: 1.1819733	best: 1.1819733 (0)	total: 56.3ms	remaining: 2m 48s
200:	learn: 1.0640221	test: 1.0522572	best: 1.0522572 (200)	total: 11.6s	remaining: 2m 41s
400:	learn: 1.0476615	test: 1.0378508	best: 1.0378508 (400)	total: 22.8s	remaining: 2m 27s
600:	learn: 1.0399211	test: 1.0326006	best: 1.0326006 (600)	total: 33.9s	remaining: 2m 15s
800:	learn: 1.0345471	test: 1.0298604	best: 1.0298604 (800)	total: 44.8s	remaining: 2m 3s
1000:	learn: 1.0297241	test: 1.0278080	best: 1.0278080 (1000)	total: 55.8s	remaining: 1m 51s
1200:	learn: 1.0249414	test: 1.0262644	best: 1.0262644 (1200)	total: 1m 6s	remaining: 1m 39s
1400:	learn: 1.0202616	test: 1.0251082	best: 1.0250985 (1397)	total: 1m 17s	remaining: 1m 28s
1600:	learn: 1.0158677	test: 1.0240951	best: 1.0240878 (1598)	total: 1m 28s	remaining: 1m 17s
1800:	learn: 1.0116655	test: 1.0231540	best: 1.0231540 (1800)	total: 1m 39s	remaining: 1m 6s
2000:	learn: 1.0074044	test: 1.0222016	best: 1.0221957 (1999)	to

Predict fold 5:   0%|          | 0/10 [00:00<?, ?it/s]


fold_rmse: [1.0165332683284727, 1.0331484794498649, 1.0226597596212912, 1.0577151482642992, 1.019600057844265]
mean_rmse: 1.0299313427016386
oof_rmse: 1.0300402189216311


In [33]:
import numpy as np
import pandas as pd

# предполагаем, что есть:
# - test (оригинальный тестовый df)
# - test_pred (предсказания из k-fold)
# - target_col (имя таргета, как в train)

id_col = "id"  # или "ID", или другое
for c in ["id", "ID", "Id"]:
    if c in test.columns:
        id_col = c
        break

if id_col is not None:
    sub = pd.DataFrame({
        id_col: test[id_col].values,
        target_col: test_pred
    })
else:
    sub = pd.DataFrame({
        "id": np.arange(len(test_pred)),
        target_col: test_pred
    })

sub.to_csv("submission_catboost_kfold_autofeat_gpu.csv", index=False)
sub.head()

,id,target
0,21472,3.420300
1,9837,3.249490
2,41791,3.861935
3,18441,3.117375
4,49348,2.765881


In [34]:
train_ag = X_tr_feat.copy()
train_ag[target_col] = y

test_ag = X_te_feat.copy()

print(train_ag.shape, test_ag.shape)
train_ag.head()

(41105, 917) (9276, 916)


,id,traffic_300m,homes_300m,works_300m,female_300m,train_ticket_order_300m,mortgage_300m,recipes_300m,online_shops_300m,manga_300m,...,__nlp__.юбилейный,__nlp__.юго,__nlp__.южная,__nlp__.южная ул,__nlp__.южнобутовская,__nlp__.южнопортовая,__nlp__.южный,__nlp__.ярославское,__nlp__._total_,target
0,1365,75429,16113.582471,15756.246444,51316.0,734.0,35.0,4820.0,10943.0,120.0,...,0,0,0,0,0,0,0,0,8,4.1
1,8230,246535,8578.458740,31315.672794,192547.0,4701.0,305.0,19354.0,53112.0,561.0,...,0,0,0,0,0,0,0,0,4,3.6
2,29071,83490,12650.492399,12490.096776,56045.0,1204.0,53.0,5093.0,12265.0,108.0,...,0,0,0,0,0,0,0,0,5,3.5
3,22591,43421,6413.279217,4963.058046,28705.0,540.0,42.0,2936.0,7495.0,70.0,...,0,0,0,0,0,0,0,0,8,4.0
4,27621,155094,18638.410128,19868.190570,123492.0,2411.0,95.0,11066.0,28705.0,237.0,...,0,0,0,0,0,0,0,0,3,4.2


In [35]:
predictor = TabularPredictor(
    label=target_col,
    problem_type="regression",
    eval_metric="rmse"
).fit(
    train_data=train_ag,
    time_limit=1800,  # можно 600/900/3600 по ситуации
    presets="medium_quality_faster_train",
    ag_args_fit={"num_gpus": 1},
)


No path specified. Models will be saved in: "AutogluonModels/ag-20251116_173254"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Dec  9 15:00:52 UTC 2
CPU Count:          62
Memory Avail:       429.29 GB / 503.46 GB (85.3%)
Disk Space Avail:   128.52 GB / 130.00 GB (98.9%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/workspace/121/AutogluonModels/ag-20251116_173254"
Train Data Rows:    41105
Train Data Columns: 916
Label Column:       target
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPi

In [36]:
id_col = "id"  # или "ID", или другое имя, если нужно

ag_pred = predictor.predict(test_ag)

sub_ag = test[[id_col]].copy()
sub_ag[target_col] = ag_pred

sub_ag.to_csv("submission_autogluon_autofeat_id_target.csv", index=False)
sub_ag.head()


,id,target
0,21472,3.371200
1,9837,3.245652
2,41791,4.003885
3,18441,3.178790
4,49348,2.697172


In [ ]:
ag_pred = predictor.predict(test_ag)

sub_ag = pd.DataFrame({target_col: ag_pred})
sub_ag.to_csv("submission_autogluon_autofeat_only_id.csv", index=False)
sub_ag.head()